In [1]:
from multiprocessing.connection import Listener
from multiprocessing import Process, Queue
from datetime import datetime

In [2]:
_remote_ = True

In [3]:
def reexecute_and_train(state_file, cell):
    import dill
    global _remote_
    filename = f"model-{datetime.now().isoformat()}"
    _remote_ = filename
    # globals()[func_name] = globals()[func_name].original
    # globals()[func_name].original
    dill.load_module(state_file)
    # if globals().get("_remote_"):
    #     print("remote = True")
    # print("executing", cell)
    exec(cell)
    # dill.dump_module(filename)
    # exec("model = train(model)")
    del _remote_
    return filename

In [4]:
def execute_in_subprocess(state_file, cell):
    def reexecute_and_train_in_subprocess(state_file, cell, queue):
        filename = reexecute_and_train(state_file, cell)
        queue.put(filename)
    
    queue = Queue()
    p = Process(target=reexecute_and_train_in_subprocess, args=(state_file, cell, queue))
    p.start()
    p.join() # this blocks until the process terminates
    filename = queue.get()
    return filename

In [5]:
from multiprocessing.connection import Listener
from multiprocessing import Process, Queue

address = ('localhost', 6001)     # family is deduced to be 'AF_INET'
listener = Listener(address, authkey=b'secret password')
try:
    while True:
        conn = listener.accept()
        state_file = conn.recv()
        cell = conn.recv()
        filename = execute_in_subprocess(state_file, cell)
        conn.send(filename)
finally:
    listener.close()

Epoch: 0 | Loss: 1.6425 | Train Accuracy: 82.27
Epoch: 1 | Loss: 1.5125 | Train Accuracy: 95.23
Epoch: 2 | Loss: 1.4834 | Train Accuracy: 98.04
